In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

data_no_additional_feature = 'training_users.csv'
data_with_tweet_csv = 'training_user_tweet.csv'
tweets_user='tweets.csv'

In [27]:
from textblob import TextBlob
import pandas as pd
from nltk.corpus import stopwords
from math import factorial
import itertools as it
from numpy import linalg as LA

#%%
def modify_df(user_tweets):
    """
    remove unnecessary words from the user_tweets.csv
    
    Argument: user_tweets
    
    Return: user_tweets dataframe
    """

    special_remove = [str(tweets) for tweets in user_tweets['text']] 
    user_tweets['tweet_split'] = [tweets.lower().split() for tweets in special_remove]
    user_tweets['tweet_split'] = [filter(lambda x: not (x.startswith("@") or x.startswith("#") or x.startswith("https:") or x in stopwords.words("english") or x.startswith("rt") or x[0].isdigit()), tweet) for tweet in user_tweets['tweet_split']]
    user_tweets['tweet_split_string'] = [' '.join(str(x) for x in tweets) for tweets in user_tweets['tweet_split']]
    return user_tweets

#%%
def comb_2(user_tweets):
    """
    calculate the number of tweet combinations
    
    Argument: user_tweets
    
    Return: total number of tweet combinations
    """
    num_tweets = len(user_tweets['text'])
    return int(factorial(num_tweets) / (factorial(2) * factorial(num_tweets - 2)))

#%%
def sim_formula(user_tweets):
    """
    calculate set of pair in tweets
    
    Argument: user_tweets
    
    Return: set of pair in tweets ratio
    """
    if len(user_tweets) != 0: 
        user_tweets = modify_df(user_tweets)
        ind = [TextBlob(tweets).word_counts for tweets in user_tweets['tweet_split_string']]
        vector_df = pd.DataFrame(ind)
        vector_df = vector_df.fillna(0)
        vector_matrix = vector_df.as_matrix()
        idx = list(it.combinations(range(vector_df.shape[0]), 2))
        sim_dot = {}
        for i, j in idx:
            sim_dot[(i, j)] = vector_matrix[i, :].dot(vector_matrix[j,:])

        sim_norm = {}
        for x, y in idx:
            sim_norm[(x, y)] = LA.norm(vector_matrix[x, :]) * LA.norm(vector_matrix[y,:])
        
        dot_set = set(sim_dot)
        norm_set = set(sim_norm)

        sim_result = {}
        for key in dot_set.intersection(norm_set):
            if sim_norm[key] != 0:
                sim_result[key] = (sim_dot[key] / sim_norm[key])/comb_2(user_tweets)
        result_sum = sum(sim_result.values())
    else:
        result_sum = "None"

    return result_sum

In [16]:
import pandas as pd
#import similarity_2 as sim_2
from pandas.io.common import CParserError
import numpy as np

#%%
sample_data_id=pd.read_csv('tweets.csv')
#sample_data_id = pd.read_csv(open("tweets.csv", 'rU'), encoding = 'utf-8', usecols = ['id'])
sample_data_id['user_id'] = sample_data_id['user_id'].astype(float)
sample_id = sample_data_id['user_id'].unique()

#%%


/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
len(sample_id)

3202

In [30]:
#filter a user
#compute similarity for all tweets in that
similarity_ratio_2_df = pd.DataFrame(columns = ["id", "similarity_ratio_2"])

i=0
for user in sample_id:
    user_df=sample_data_id.loc[sample_data_id['user_id'] == user]
    similarity_ratio_2=sim_formula(user_df)
    #similarity_ratio_2 = [sim_formula(sample_tw[int(i)]) for i in range(len(sample_tw))]
    similarity_ratio_2_df.loc[i] = [user,similarity_ratio_2]
    i=i+1
    

    
    

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

KeyboardInterrupt: 

In [31]:
similarity_ratio_2_df

,id,similarity_ratio_2
0,10935572.0,0.036579
1,16119337.0,0.067707
2,16753788.0,0.050538
3,17640121.0,0.039088
4,17656600.0,0.032042
5,19230427.0,0.013791
6,19951698.0,0.034230
7,20731719.0,0.180602
8,21196037.0,0.017680
9,21375100.0,0.012513


In [52]:
from __future__ import division
"""
verify tweet‘s authenticity using url_ratio, url_unique_ratio, hashtag_ratio, username_ratio and username_unique_ratio
"""

#from compiler.ast import flatten
import re
import collections

def flatten(l):
    for el in l:
        if isinstance(el, collections.Iterable) and not isinstance(el, (str, bytes)):
            yield from flatten(el)
        else:
            yield el
            
#%%
def url_ratio(user_tweets):
    """
    calculate the percentage of 20 recent Tweets containing URLs
    
    Argument: tweets_df
    
    Return: tweets_url_ratio
    """
    if len(user_tweets) != 0: 
        top_20 = user_tweets[:20]
        tweets_url_ratio = sum(top_20['text'].str.contains("https:") == True)/len(user_tweets['text'])
    else:
        tweets_url_ratio = 'None'
    return tweets_url_ratio


#%%
def url_unique_ratio(user_tweets):
    """
    calculate the ratio of the number of unique URLs in the 20 recent tweets
    
    Argument: tweets_df
    
    Return: url_ratio
    """
    if len(user_tweets) != 0: 
        top_20 = user_tweets[:20]
        # find all the urls using regular expression
        urls = [re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', str(tweets)) for tweets in top_20['text']]
        # flatten a list of lists
        urls_flatten = flatten(urls)
        #urls_flatten = urls.flatten()
        # get the first two parts of the url
        urls_split = [u.split('/')[0:3] for u in urls_flatten]
        
        urls_unique = [list(u) for u in set(tuple(u) for u in urls_split)]
        url_unique= len(urls_unique)
        tweet_total = len(user_tweets['text'])
        url_ratio = url_unique/tweet_total
    else:
        url_ratio = 'None'
    return url_ratio

#%%
def hashtag_ratio(user_tweets):
    """
    calculate the hashtag ratio
    
    Argument: tweets_df
    
    Return: hashtag ratio
    """
    if len(user_tweets) != 0: 
        top_20 = user_tweets[:20]
        hashtag_ratio = sum(top_20['text'].str.contains("#"))/len(top_20['text'])
    else:
        hashtag_ratio = 'None'
    return hashtag_ratio

#%%
def username_ratio(user_tweets):
    """
    calculate the username ratio
    
    Argument: tweets_df
    
    Return: username ratio
    """
    if len(user_tweets) != 0: 
        top_20 = user_tweets[:20]
        username_ratio = sum(top_20['text'].str.contains("@"))/len(top_20['text'])
    else:
        username_ratio = 'None'
    return username_ratio
    
#%%
def username_unique_ratio(user_tweets):
    """
    calculate the ratio of the number of unique @usernames
    
    Argument: tweets_df
    
    Return: username_unique_ratio
    """
    if len(user_tweets) != 0: 
        top_20 = user_tweets[:20]
    
        username = [re.findall('@([A-Za-z0-9_]+)', str(tweets)) for tweets in top_20['text']]
        # flatten a list of lists
        username_flatten = flatten(username)
        #username_flatten = username.flatten()
        username_unique = set(username_flatten)
        user_unique= len(username_unique)
        # total number of users that were being @, not all the tweets
        tweet_total = len(user_tweets['text'])
        user_ratio = user_unique/tweet_total
    else:
        user_ratio = 'None'
    return user_ratio

In [53]:
sample_tweet_analysis_ratio = pd.DataFrame(columns = ["id", "url_ratio", "url_unique_ratio", "hashtag_ratio", "username_ratio", "username_unique_ratio"])

i=0
for user in sample_id:
    t_df=sample_data_id.loc[sample_data_id['user_id'] == user]
    
    url_ratio_out = url_ratio(t_df)

    #%%
    url_unique_ratio_out = url_unique_ratio(t_df)

    #%%
    hashtag_ratio_out = hashtag_ratio(t_df)

    #%%
    username_ratio_out = username_ratio(t_df)

    #%%
    username_unique_ratio_out = username_unique_ratio(t_df)
    #similarity_ratio_2 = [sim_formula(sample_tw[int(i)]) for i in range(len(sample_tw))]
    sample_tweet_analysis_ratio.loc[i] = [user,url_ratio_out, url_unique_ratio_out, hashtag_ratio_out, username_ratio_out, username_unique_ratio_out]
    i=i+1

In [54]:
sample_tweet_analysis_ratio

,id,url_ratio,url_unique_ratio,hashtag_ratio,username_ratio,username_unique_ratio
0,1.093557e+07,0.001799,0.003597,0.000000,0.500000,0.017986
1,1.611934e+07,0.000000,0.000000,0.950000,0.050000,0.002817
2,1.675379e+07,0.000000,0.233333,0.000000,0.300000,0.200000
3,1.764012e+07,0.000000,0.050000,0.250000,0.800000,0.750000
4,1.765660e+07,0.000000,0.003268,0.200000,0.850000,0.071895
5,1.923043e+07,0.000000,0.010417,0.400000,0.700000,0.083333
6,1.995170e+07,0.000000,0.005181,0.400000,0.000000,0.000000
7,2.073172e+07,0.000000,0.002294,0.000000,0.400000,0.018349
8,2.119604e+07,0.000000,0.012821,0.050000,0.200000,0.064103
9,2.137510e+07,0.000000,0.004587,0.300000,0.700000,0.068807


In [ ]:
#statuses count
#ff_ratio
#tweet rate
def tweet_rate(data):
    dates = [i.rsplit()[0] for i in data['created_at']]
    total_dates = len(set(dates))
    tweet_rate = len(dates)/total_dates
    
    return tweet_rate
